# Beat The Bookies: Predicting EPL Matches
_Team C_

__Mohammad Ali Syed, Abdul Al-Fahim, Dylan Hoi, Henry Chen, Chris Wong & Yolanne Lee__

**Contents:**

[Section 1](#section1): Introduction

[Section 2](#section2): Data Import

[Section 3](#section3): Data Transformation & Exploration

[Section 4](#section4): Methodology Overview

[Section 5](#section5): Model Training & Validation

[Section 6](#section6): Results

[Section 7](#section7): Final Predictions on Test Set

## 1. Introduction

## 2. Data Import
<a name='section2'></a>

In [ ]:
#Import packages
import math
import numpy as np
import pandas as pd
import datetime as datetime
import seaborn as sns
from collections import Counter, deque

#!pip install geopy
#!pip install sklearn

#For Computing Priors
from geopy.distance import geodesic 
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, classification_report,confusion_matrix, accuracy_score


#For Visualisation
from sklearn.tree import export_graphviz
from subprocess import call
from IPython.display import Image

#For Model Selection
from sklearn.model_selection import RepeatedStratifiedKFold, KFold, RepeatedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt


In [ ]:
# Import Data

# EPL Training Data
dirName = 'Data_Files/'
filePath = dirName + 'epl-training.csv'
data = pd.read_csv(filePath)

# Additional EPL Training Data
# downloaded from www.football-stats.co.uk and concatenated from seasons 2000-2008.
# Reformatted to suit our current data architecture, additional 3,047 rows x 22 columns
filePath = dirName + 'epl-training-extra.csv'
extraData = pd.read_csv(filePath)
#data = data.append(extraData, ignore_index = True) #append additional data

# Additional EPL Stadium Location Data
filePath = dirName + 'epl-stadium.csv'
locationData = pd.read_csv(filePath)

# Additional EPL Goalkeeper Data
filePath = dirName + 'epl-goalkeeping.csv'
GKData = pd.read_csv(filePath)

#Remove empty nan columns at the end
data = data.iloc[:, 0:22]
pd.set_option('display.max_columns', None)
data.head()

## 3. Data Transformation & Exploration
<a name='section3'></a>

In [ ]:
#Helper Functions

def corr_matrix(X, feature):
    corr= X.corr()
    corr_y = abs(corr[feature])
    highest_corr = corr_y[corr_y >0.2]
    highest_corr.sort_values(ascending=True)
    return highest_corr

def rf_model(X_train, X_test, y_train, y_test):
    rf=RandomForestClassifier(random_state = 42)
    rf.fit(X_train, y_train)
    preds = rf.predict(X_test)
    accuracy = calc_accuracy(preds, y_test)
    return rf, preds, accuracy

def feat_importances(X_train, rf):
    feature_importances = list(zip(X_train, rf.feature_importances_))
    feature_importances_ranked = sorted(feature_importances, key = lambda x: x[1], reverse = True)
    return feature_importances_ranked

def select_feat(X_train, y_train):
    feature_selector = SelectFromModel(RandomForestClassifier(random_state = 42)).fit(X_train, y_train)
    selected_feat= X_train.columns[(feature_selector.get_support())]
    return selected_feat

def calc_accuracy(preds, labels):
    accuracy = accuracy_score(labels, preds) * 100
    return accuracy

def rf_tree_visualiser(rf, featuresetName, feature_names):
    tree = rf.estimators_[10]  #Take 10th random tree
    export_graphviz(tree, out_file = featuresetName + '.dot', feature_names = list(feature_names),
                    rounded = True, proportion = False, 
                    precision = 2, filled = True, max_depth = 3)
    call(['dot', '-Tpng', featuresetName + '.dot', '-o', featuresetName + '.png'],shell=True)
    return featuresetName + '.png'

def scatter(data, title, xlabel, ylabel):
    # Assume data is an array of tuples
    x, y = zip(*data)
    # s is the area of the circles in the plot
    plt.scatter(x, y, s=50)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.show()
    
# https://towardsdatascience.com/stop-one-hot-encoding-your-time-based-features-24c699face2f
def transformation(column):
    max_value = column.max()
    sin_values = [math.sin((2*math.pi*x)/max_value) for x in list(column)]
    cos_values = [math.cos((2*math.pi*x)/max_value) for x in list(column)]
    return sin_values, cos_values

## 3.1 Intial Data Exploration

In [ ]:
############################################# Feature Visualisation
# Visualise correlations between different statistics
from pandas.plotting import scatter_matrix

# Sort data by teams
teams = {}
referees = {}
for i in data.groupby('HomeTeam').mean().T.columns:
    teams[i] = []
for i in data.groupby('Referee').mean().T.columns:
    referees[i] = []

# Team Summary Statistics
home_team_stats = pd.DataFrame()
away_team_stats = pd.DataFrame()

teams = pd.unique(data[["HomeTeam"]].values.ravel())

for team in teams:
    # Compute summary stats as home team
    team_stats = data[(data["HomeTeam"] == team)]
    team_stats = team_stats.iloc[:, [3, 6, 10, 12, 14, 16, 18, 20]]
    team_stats = team_stats.sum()

    performance = data[(data["HomeTeam"] == team)].iloc[:, 5]
    num_vals = len(performance)
    
    performance = performance.value_counts()
    performance_keys = performance.keys()
    performance_values = performance.values
    performance = zip(performance.keys(), performance.values)
    
    for key, value in performance:
        metric = value/num_vals
        
        if key == "H":
            team_stats["Win Rate"] = metric
            
        elif key == "A":
            team_stats["Lose Rate"] = metric
        
        else:
            team_stats["Draw Rate"] = metric

    home_team_stats[team] = pd.DataFrame(team_stats) ##causing problems

    # Compute summary stats as away team
    team_stats = data[(data["AwayTeam"] == team)]
    team_stats = team_stats.iloc[:, [4, 7, 11, 13, 15, 17, 19, 21]]
    team_stats = team_stats.sum()

    performance = data[(data["AwayTeam"] == team)].iloc[:, 5]
    num_vals = len(performance)

    performance = performance.value_counts()
    performance_keys = performance.keys()
    performance_values = performance.values
    performance = zip(performance.keys(), performance.values)
    
    for key, value in performance:
        metric = value/num_vals
        
        if key == "A":
            team_stats["Win Rate"] = metric
            
        elif key == "H":
            team_stats["Lose Rate"] = metric
        
        else:
            team_stats["Draw Rate"] = metric


    away_team_stats[team] = pd.DataFrame(team_stats)

# Sort by strongest to weakest team, by win rate
home_team_stats = home_team_stats.sort_values(by='Win Rate', axis=1, ascending=False)
away_team_stats = away_team_stats.sort_values(by='Win Rate', axis=1, ascending=False)
home_team_stats
#Interesting to note, Man U ranked lower on every metric except fouls and yellow cards compared to Chelsea but had higher win rate -> could suggest the more aggressive the team, the higher the win rate
# print(home_team_stats.iloc[:, 0])
# print(away_team_stats.iloc[:, 0])
# print(np.array(home_team_stats.iloc[:, 0]) - np.array(away_team_stats.iloc[:, 0]))

### 3.1.2 Relationship Between Attributes

In [ ]:
#Correlation matrix between full time goals and other features
highest_corr = corr_matrix(data, "FTHG")
print("FTHG: \n" + str(highest_corr))

highest_corr = corr_matrix(data, "FTAG")
print("FTAG: \n" + str(highest_corr))

In [ ]:
#Split dataset into input and output data

#Output variable
y = data.iloc[:, 5:6]
#Reformat y to make it suitable for LabelEncoder

y = np.array(y).reshape(len(y))
# #Encode y
# y = LabelEncoder().fit_transform(y) #################this needs to be done separately for train/test

#Input variables
#Remove give away columns such as goals scored
data_filtered = data.drop(labels = data.columns[[3, 4, 5, 6, 7, 8]], axis = 1)

In [ ]:
#Data preprocessing

#Dates
data_filtered['Date'] = pd.to_datetime(data_filtered['Date'])
#year has been removed as we need to predict future results -> https://towardsdatascience.com/machine-learning-with-datetime-feature-engineering-predicting-healthcare-appointment-no-shows-5e4ca3a85f96
year = data_filtered['Date'].dt.year
data_filtered['Month'] = data_filtered['Date'].dt.month
data_filtered['Week'] = data_filtered['Date'].dt.isocalendar().week
data_filtered['Day'] = data_filtered['Date'].dt.day
#Extract encoded dates
dates_split = data_filtered.iloc[:, 16:19]
#Remove encoded dates and original date column
data_filtered = data_filtered.drop(labels = data_filtered.columns[[0, 16, 17, 18]], axis = 1)

#Encode categorical data
encoder = OneHotEncoder(handle_unknown='ignore')

#Teams
home_t = data_filtered.iloc[:, 0:1]
home_t = encoder.fit_transform(home_t) #################does this need to be done separately?

away_t = data_filtered.iloc[:, 1:2]
away_t = encoder.fit_transform(away_t) #################does this need to be done separately?
data_filtered = data_filtered.drop(labels = data_filtered.columns[[0,1]], axis = 1)

#Referees 
ref = data_filtered.iloc[:, 0:1]
ref = encoder.fit_transform(ref)       #################does this need to be done separately?
data_filtered = data_filtered.drop(labels = data_filtered.columns[[0]], axis = 1)

#Re-stack columns
data_filtered = data_filtered.join(pd.DataFrame(ref.toarray()), rsuffix = '_ref')
data_filtered = data_filtered.join(pd.DataFrame(home_t.toarray()), rsuffix = '_home')
data_filtered = data_filtered.join(pd.DataFrame(away_t.toarray()), rsuffix = '_away')
data_filtered = dates_split.join(data_filtered)
data_filtered.columns = data_filtered.columns.astype(str)
data_filtered.head()

In [ ]:
#Train model on entire featureset
X_train, X_test, y_train, y_test = train_test_split(data_filtered, y, test_size=0.3, random_state=42)

y_train = np.array(y_train).reshape(len(y_train))
y_test = np.array(y_test).reshape(len(y_test))
#Encode y
encoder = LabelEncoder().fit(y_train)
y_train = encoder.transform(y_train)
y_test = encoder.transform(y_test)

rf, preds, base_accuracy = rf_model(X_train, X_test, y_train, y_test)
print("Accuracy on entire featureset: " + str(base_accuracy) + "%")


In [ ]:
#Print rf tree N.B. may not work without importing graphviz, random forest images will be on GitHub
Image(filename = rf_tree_visualiser(rf, 'featureSetTree', data_filtered.columns))

In [ ]:
#Train model without Referee feature
data_filtered_no_ref = data_filtered.iloc[:, 0:15].join(data_filtered.iloc[:, 58:])
X_train, X_test, y_train, y_test = train_test_split(data_filtered_no_ref, y, test_size=0.3, random_state=42)

y_train = np.array(y_train).reshape(len(y_train))
y_test = np.array(y_test).reshape(len(y_test))
#Encode y
encoder = LabelEncoder().fit(y_train)
y_train = encoder.transform(y_train)
y_test = encoder.transform(y_test)


rf, preds, accuracy = rf_model(X_train, X_test, y_train, y_test)
print("Accuracy without Referee: " + str(accuracy) + "%")
print("Difference from before: " + str(accuracy - base_accuracy) + "%")
#Ref is having negative impact so remove
data_filtered = data_filtered_no_ref

In [ ]:
#Print rf tree (no ref)
Image(filename = rf_tree_visualiser(rf, 'featureSetTreeNoRef', data_filtered_no_ref.columns))

In [ ]:
#Train model without Date feature
data_filtered_no_date = data_filtered.iloc[:, 3:]
X_train, X_test, y_train, y_test = train_test_split(data_filtered_no_date, y, test_size=0.3, random_state=42)

y_train = np.array(y_train).reshape(len(y_train))
y_test = np.array(y_test).reshape(len(y_test))
#Encode y
encoder = LabelEncoder().fit(y_train)
y_train = encoder.transform(y_train)
y_test = encoder.transform(y_test)


rf, preds, accuracy = rf_model(X_train, X_test, y_train, y_test)
print("Accuracy without Dates: " + str(accuracy) + "%")
print("Difference from before: " + str(accuracy - base_accuracy) + "%")

In [ ]:
#Print rf tree (no dates)
Image(filename = rf_tree_visualiser(rf, 'featureSetTreeNoDate', data_filtered_no_date.columns))

In [ ]:
#Train model on only in-game stats to identify most important ones
data_filtered_only_game_stats = data_filtered.iloc[:, 3:15]
X_train, X_test, y_train, y_test = train_test_split(data_filtered_only_game_stats, y, test_size=0.3, random_state=42)

y_train = np.array(y_train).reshape(len(y_train))
y_test = np.array(y_test).reshape(len(y_test))
#Encode y
encoder = LabelEncoder().fit(y_train)
y_train = encoder.transform(y_train)
y_test = encoder.transform(y_test)


rf, preds, all_stats_accuracy = rf_model(X_train, X_test, y_train, y_test)
print("Accuracy on all in-game stats: " + str(all_stats_accuracy) + "%")

In [ ]:
#Visualise and analyse initial results

#Display feature importances in descending order
feature_importances = feat_importances(X_train, rf)
print("Feature Importances: ")
[print('Feature: {:35} Importance: {}'.format(*pair)) for pair in feature_importances];

print("\nConfusion Matrix: ")
print(confusion_matrix(y_test, preds))
print("\nClassification Report: ")
print(classification_report(y_test, preds))
#Important note: AF/HF rank higher than HC/AC

In [ ]:
# Visualise feature importance
scatter(feature_importances, "Feature importances", "Feature", "Importance")

In [ ]:
# plot Pearson Correlation Heatmap to see the top 10 features related to the match result FTR

def plotGraph(X_all, Y_all):

    train_data=pd.concat([X_all,Y_all],axis=1)

    #FTR correlation matrix
    plt.figure(figsize=(12,12))
    k = 11 # number of variables for heatmap
    cols = abs(train_data.astype(float).corr()).nlargest(k, 'FTR')['FTR'].index
    cm = np.corrcoef(train_data[cols].values.T)
    sns.set(font_scale=1.25)
    hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 12}, yticklabels=cols.values, xticklabels=cols.values)
    plt.show()

attributes = data.drop(['Date','HomeTeam', 'AwayTeam', 'Referee','FTR'],1)
attributes['HTR'] = attributes['HTR'].map({'H':1,'A':0,'D':2})
label = data['FTR']
label = label.map({'H':1,'A':0,'D':2})
plotGraph(attributes,label)


In [ ]:
#Feature Selection
#change names and display selected features more nicely, ideally with their importance, gini impurity...
selected_feat = select_feat(X_train, y_train)
print(selected_feat)

In [ ]:
#Train model on selected in-game stats only
indexes = []
for feat in selected_feat:
    indexes.append(data_filtered_only_game_stats.columns.get_loc(feat))
    
data_filtered_filtered_game_stats = data_filtered_only_game_stats.iloc[:, indexes]

X_train, X_test, y_train, y_test = train_test_split(data_filtered_filtered_game_stats, y, test_size=0.3, random_state=42)

y_train = np.array(y_train).reshape(len(y_train))
y_test = np.array(y_test).reshape(len(y_test))
#Encode y
encoder = LabelEncoder().fit(y_train)
y_train = encoder.transform(y_train)
y_test = encoder.transform(y_test)


rf, preds, reduced_stats_accuracy = rf_model(X_train, X_test, y_train, y_test)
print("Accuracy on reduced in-game stats: " + str(reduced_stats_accuracy) + "%")
print("Difference compared to all in-game stats: " + str(reduced_stats_accuracy - all_stats_accuracy) + "%")

print("\nConfusion Matrix: ")
print(confusion_matrix(y_test, preds))
print("\nClassification Report: ")
print(classification_report(y_test, preds))

In [ ]:
#Visualisation of new featureset/tree
data_filtered_filtered_game_stats.plot(kind='hist', subplots=True, sharex=False, sharey=False, bins=50, layout=(2,4), figsize=(12, 6))
data_filtered_filtered_game_stats.plot(kind='box', subplots=True, layout=(2,4), sharex=False, sharey=False, figsize=(12, 6))
data_filtered_filtered_game_stats.plot(kind='density', subplots=True, layout=(2,4), sharex=False, sharey=False, figsize=(12, 6))
Image(filename = rf_tree_visualiser(rf, 'selectedFeatureSetTree', data_filtered_filtered_game_stats.columns))

In [ ]:
#Produce new dataset
#Fix column names
#Restack teams and dates

#Original teams are needed to be able to compute priors
data_new = data.iloc[:, [1, 2]].join(data_filtered_filtered_game_stats)
data_new = dates_split.join(data_new)

#Stack previously removed giveaway columns
data_new = data_new.join(data.iloc[:, [3, 4, 5, 6, 7, 8]])

#Feature engineer second half goals
#Second half home goals
SHHG = np.array(data.iloc[:, [3]]) - np.array(data.iloc[:, [6]])
#Second half away goals
SHAG = np.array(data.iloc[:, [4]]) - np.array(data.iloc[:, [7]])
data_new['SHHG'] = pd.DataFrame(SHHG)
data_new['SHAG'] = pd.DataFrame(SHAG)
data_new.columns = data_new.columns.astype(str)
data_new.head()

In [ ]:
#See if second half goals have significant correlation to total goals
highest_corr = corr_matrix(data_new, "FTHG")
print("FTHG: \n" + str(highest_corr))

highest_corr = corr_matrix(data_new, "FTAG")
print("FTAG: \n" + str(highest_corr))
#Second half goals do have very strong correlation

## 3.2 Priors Feature Construction

In [ ]:
# From Pearson Correlation Heatmap to extract the top 10 features 
# there are two pairs of data highly correlated (see details in report), 
# so we just pick [FTHG, FTAG, HS, AS, HR, AR] from the top 10 features,
# additionally [Date, HomeTeam, AwayTeam, FTR], to derive our features.
selectedAttributes = ["Date","HomeTeam", "AwayTeam","FTR","FTHG","FTAG","HS","AS","HR","AR"]
training_data = data[selectedAttributes]

### 3.2.1 Data Cleaning

In [ ]:
#Derive features and remove unwanted data
def removeInvalidData(data):

    # remove data which contains None
    data.dropna(axis=0, how='any',inplace=True)

    # remove data which contains NaN, infinite or overflowed number 
    indices_to_keep = ~data.isin([np.nan, np.inf, -np.inf]).any(1)
    data = data[indices_to_keep]

    return data

#check if there are rows containing None, NaN, infinite or overflowed values
assert data.shape[0] == removeInvalidData(data).shape[0]
data = removeInvalidData(data)


### 3.2.2 Cumulative Full-time W/L Ratio

In [ ]:
# Computing Priors
# Calculate cumulative Full-Time win-loss ratio for Home/Away teams prior to every match
# TODO: Points-based results based on previous wins & losses 
# PHWL = Previous Home Team Win Loss Ratio
# PAWL = Previous Away Team Win Loss Ratio

def get_previousFTResults(playing_stat):
    
    # Create a dictionary with team names as keys
    teams = {}
    PHWL = []
    PAWL = []
    
    for i in playing_stat.groupby('HomeTeam').mean().T.columns:
        teams[i] = [] #Each team gets their own list

    # the value corresponding to keys is a list containing the match result
    for i in range(len(playing_stat)):
        
        #list of respective Home/Away team in match
        match_ht = teams[playing_stat.iloc[i].HomeTeam]
        match_at = teams[playing_stat.iloc[i].AwayTeam]
        
        #count no. of wins
        
        h_wins = Counter(match_ht)
        a_wins = Counter(match_at)
        
        #h_wins = no. of home wins
        #a_wins = no. of away wins
        h_wins = h_wins['W']
        a_wins = a_wins['W']
        
        #append W/L/D to respective teams
        
        if playing_stat.iloc[i].FTR == 'H':
            match_ht.append('W')
            match_at.append('L')
        elif playing_stat.iloc[i].FTR == 'A':
            match_at.append('W')
            match_ht.append('L')
        else:
            match_at.append('D')
            match_ht.append('D')
       
        h_wlRatio = h_wins / len(match_ht)
        a_wlRatio = a_wins / len(match_at)
        
        #Home/Away cumulative WL ratios prior to every match
        PHWL.append(h_wlRatio)
        PAWL.append(a_wlRatio)
        
    data_new.loc[:,'PHWL'] = pd.Series(PHWL)
    data_new.loc[:,'PAWL'] = pd.Series(PAWL)

    return data_new
#get_previousFTResults(data_new)


###  3.2.3 Cumulative Half-time W/L Ratio

In [ ]:
# Computing Priors
# Calculate cumulative Half-Time win-loss ratio for Home/Away teams prior to every match
# HHTR = Previous Home Half Time Results
# AHTR = Previous Away Half Time Results

def get_PreviousHTResults(playing_stat):
    
    # Create a dictionary with team names as keys
    teams = {}
    HHTR = []
    AHTR = []
    
    for i in playing_stat.groupby('HomeTeam').mean().T.columns:
        teams[i] = [] #Each team gets their own list

    # the value corresponding to keys is a list containing the match result
    for i in range(len(playing_stat)):
        
        #list of respective Home/Away team in match
        match_ht = teams[playing_stat.iloc[i].HomeTeam]
        match_at = teams[playing_stat.iloc[i].AwayTeam]
        
        #count no. of wins
        
        h_wins = Counter(match_ht)
        a_wins = Counter(match_at)
        
        #h_wins = no. of home wins
        #a_wins = no. of away wins
        h_wins = h_wins['W']
        a_wins = a_wins['W']
        
        #append W/L/D to respective teams
        
        if playing_stat.iloc[i].HTR == 'H':
            match_ht.append('W')
            match_at.append('L')
        elif playing_stat.iloc[i].HTR == 'A':
            match_at.append('W')
            match_ht.append('L')
        else:
            match_at.append('D')
            match_ht.append('D')
            
        h_wlRatio = h_wins / len(match_ht)
        a_wlRatio = a_wins / len(match_at)
       
        #Home/Away cumulative WL ratios prior to every match
        HHTR.append(h_wlRatio)
        AHTR.append(a_wlRatio)
        
    data_new.loc[:,'HHTR'] = pd.Series(HHTR)
    data_new.loc[:,'AHTR'] = pd.Series(AHTR)

    return data_new


#get_PreviousHTResults(data_new)

### 3.2.4 Cumulative Full-Time goals scored

In [ ]:
# Computing Priors
# Calculate Previous Full-Time Cumulative Goal 
# PHGS = Previous Home Goal Scored
# PAGS = Previous Away Goal Scored

def getPreviousCumulativeGoals(data):
    teams = {}
    PHGS = [] 
    PAGS = []   

    # for each match
    for i in range(len(data)):
        if (i % 100000 == 0):
            for name in data.groupby('HomeTeam').mean().T.columns:
                teams[name] = [0]

        FTHG = data.iloc[i]['FTHG']
        FTAG = data.iloc[i]['FTAG']

        try:
            pcgs_h = teams[data.iloc[i].HomeTeam].pop()
            pcgs_a = teams[data.iloc[i].AwayTeam].pop()
        except:
            pcgs_h = 0
            pcgs_a = 0

        PHGS.append(pcgs_h)
        PAGS.append(pcgs_a)
#         print(PAGS)
#         print(PHGS)
        pcgs_h = pcgs_h + FTHG #Home team's previous goals scored before this match
        teams[data.iloc[i].HomeTeam].append(pcgs_h)
        pcgs_a = pcgs_a + FTAG #Away team's previous goals scored before this match
        teams[data.iloc[i].AwayTeam].append(pcgs_a)

    data_new.loc[:,'PHGS'] = pd.Series(PHGS)
    data_new.loc[:,'PAGS'] = pd.Series(PAGS)
    return data_new

#getPreviousCumulativeGoals(data_new)

### 3.2.5 Cumulative Half-time W/L Ratio

In [ ]:
# Computing Priors
# Calculate Previous Shots in the match
# PHS = Home teams previous match Shots, totaled over season
# PAS = Away teams previous match Shots, totaled over season

def getPreviousShots(data):
    teams = {}
    PHS = [] 
    PAS = []   

    # for each match
    for i in range(len(data)):
        if (i % 100000 == 0):
            for name in data.groupby('HomeTeam').mean().T.columns:
                teams[name] = [0]

        HS = data.iloc[i]['HS']
        AS = data.iloc[i]['AS']

        try:
            pcs_h = teams[data.iloc[i].HomeTeam].pop()
            pcs_a = teams[data.iloc[i].AwayTeam].pop()
        except:
            pcs_h = 0
            pcs_a = 0

        PHS.append(pcs_h)
        PAS.append(pcs_a)
        pcs_h = pcs_h + HS #Home team's previous goals scored before this match
        teams[data.iloc[i].HomeTeam].append(pcs_h)
        pcs_a = pcs_a + AS #Away team's previous goals scored before this match
        teams[data.iloc[i].AwayTeam].append(pcs_a)

    data_new.loc[:,'PHS'] = pd.Series(PHS)
    data_new.loc[:,'PAS'] = pd.Series(PAS)
    return data_new

#getPreviousShots(data_new)

### 3.2.6 Previous shots on target

In [ ]:
# Computing Priors
# Calculate Previous Shots on Target
# PHSOT = Home teams Previous Shots on Target, totaled over season
# PASOT = Away teams Previous Shots on Target, totaled over season

def getPreviousShotsOnTarget(data):
    teams = {}
    PHSOT = [] 
    PASOT = []   

    # for each match
    for i in range(len(data)):
        if (i % 100000 == 0):
            for name in data.groupby('HomeTeam').mean().T.columns:
                teams[name] = [0]

        HST = data.iloc[i]['HST']
        AST = data.iloc[i]['AST']

        try:
            pcsot_h = teams[data.iloc[i].HomeTeam].pop()
            pcsot_a = teams[data.iloc[i].AwayTeam].pop()
        except:
            pcsot_h = 0
            pcsot_a = 0

        PHSOT.append(pcsot_h)
        PASOT.append(pcsot_a)
        pcsot_h = pcsot_h + HST #Home team's previous goals scored before this match
        teams[data.iloc[i].HomeTeam].append(pcsot_h)
        pcsot_a = pcsot_a + AST #Away team's previous goals scored before this match
        teams[data.iloc[i].AwayTeam].append(pcsot_a)

    data_new.loc[:,'PHSOT'] = pd.Series(PHSOT)
    data_new.loc[:,'PASOT'] = pd.Series(PASOT)
    return data_new

#getPreviousShotsOnTarget(data_new)

### 3.2.7 Computing previous fouls

In [ ]:
# Computing Priors
# Calculate Previous Fouls
# PHTF = Home teams Previous Fouls, Totaled over season
# PATF = Away teams Previous Fouls, Totaled over season

def getPreviousTeamFouls(data):
    teams = {}
    PHTF = [] 
    PATF = []   

    # for each match
    for i in range(len(data)):
        if (i % 100000 == 0):
            for name in data.groupby('HomeTeam').mean().T.columns:
                teams[name] = [0]

        HF = data.iloc[i]['HF']
        AF = data.iloc[i]['AF']

        try:
            pcf_h = teams[data.iloc[i].HomeTeam].pop()
            pcf_a = teams[data.iloc[i].AwayTeam].pop()
        except:
            pcf_h = 0
            pcf_a = 0

        PHTF.append(pcf_h)
        PATF.append(pcf_a)
        pcf_h = pcf_h + HF #Home team's previous fouls before this match
        teams[data.iloc[i].HomeTeam].append(pcf_h)
        pcf_a = pcf_a + AF #Away team's previous fouls before this match
        teams[data.iloc[i].AwayTeam].append(pcf_a)

    data_new.loc[:,'PHTF'] = pd.Series(PHTF)
    data_new.loc[:,'PATF'] = pd.Series(PATF)
    return data_new

#getPreviousTeamFouls(data_new)

### 3.2.8 Computing previous corners

In [ ]:
# Computing Priors
# Calculate Previous Corners
# PHTC = Home teams Previous Corners, Totaled over season
# PATC = Away teams Previous Corners, Totaled over season

def getPreviousTeamCorners(data):
    teams = {}
    PHTC = [] 
    PATC = []   

    # for each match
    for i in range(len(data)):
        if (i % 100000 == 0):
            for name in data.groupby('HomeTeam').mean().T.columns:
                teams[name] = [0]

        HC = data.iloc[i]['HC']
        AC = data.iloc[i]['AC']

        try:
            pcc_h = teams[data.iloc[i].HomeTeam].pop()
            pcc_a = teams[data.iloc[i].AwayTeam].pop()
        except:
            pcc_h = 0
            pcc_a = 0

        PHTC.append(pcc_h)
        PATC.append(pcc_a)
        pcc_h = pcc_h + HC #Home team's previous corners before this match
        teams[data.iloc[i].HomeTeam].append(pcc_h)
        pcc_a = pcc_a + AC #Away team's previous corners before this match
        teams[data.iloc[i].AwayTeam].append(pcc_a)

    data_new.loc[:,'PHTC'] = pd.Series(PHTC)
    data_new.loc[:,'PATC'] = pd.Series(PATC)
    return data_new

#getPreviousTeamCorners(data_new)

### 3.2.9 Computing previous goals before half-time

In [ ]:
# Computing Priors
# Calculate Previous Goals before half time
# PHTHG = Home teams Previous Goals Before Half Time, Totaled over season
# PHTAG = Away teams Previous Goals Before Half Time, Totaled over season

def getPreviousHalfTimeGoalsScored(data):
    teams = {}
    PHTHG = [] 
    PHTAG = []   

    # for each match
    for i in range(len(data)):
        if (i % 100000 == 0):
            for name in data.groupby('HomeTeam').mean().T.columns:
                teams[name] = [0]

        HTHG = data.iloc[i]['HTHG']
        HTAG = data.iloc[i]['HTAG']

        try:
            pchtg_h = teams[data.iloc[i].HomeTeam].pop()
            pchtg_a = teams[data.iloc[i].AwayTeam].pop()
        except:
            pchtg_h = 0
            pchtg_a = 0

        PHTHG.append(pchtg_h)
        PHTAG.append(pchtg_a)
        pchtg_h = pchtg_h + HTHG #Home team's previous first half goals scored before this match
        teams[data.iloc[i].HomeTeam].append(pchtg_h)
        pchtg_a = pchtg_a + HTAG #Away team's previous first half goals scored before this match
        teams[data.iloc[i].AwayTeam].append(pchtg_a)

    data_new.loc[:,'PHTHG'] = pd.Series(PHTHG)
    data_new.loc[:,'PHTAG'] = pd.Series(PHTAG)
    return data_new

#getPreviousHalfTimeGoalsScored(data_new)

### 3.3.0 Compute previous goals after half-time

In [ ]:
# Computing Priors
# Calculate Previous Second Half Time Goals in the match
# PSHHG = Previous Second Half Time Goals scored by Home team, totaled over season
# PSHAG = Previous Second Half Time Goals scored by Away team, totaled over season

def getPreviousSecondHalfGoals(data):
    teams = {}
    PSHHG = [] 
    PSHAG = []   
    
    # for each match
    for i in range(len(data)):
        if (i % 100000 == 0):
            for name in data.groupby('HomeTeam').mean().T.columns:
                teams[name] = [0]
                
        FTHG = data.iloc[i]['FTHG']
        FTAG = data.iloc[i]['FTAG']
        HTHG = data.iloc[i]['HTHG']
        HTAG = data.iloc[i]['HTAG']

        try:
            shg_h = teams[data.iloc[i].HomeTeam].pop()
            shg_a = teams[data.iloc[i].AwayTeam].pop()
        except:
            shg_h = 0
            shg_a = 0

        PSHHG.append(shg_h)
        PSHAG.append(shg_a)
        shg_h = shg_h + (FTHG - HTHG) #Home team's previous second half goals scored before this match
        teams[data.iloc[i].HomeTeam].append(shg_h)
        shg_a = shg_a + (FTAG - HTAG) #Away team's previous second half goals scored before this match
        teams[data.iloc[i].AwayTeam].append(shg_a)

    data_new.loc[:,'PSHHG'] = pd.Series(PSHHG)
    data_new.loc[:,'PSHAG'] = pd.Series(PSHAG)
    return data_new

#getPreviousSecondHalfGoals(data_new)

### 3.3.1 Computing previous goals conceded before half-time

In [ ]:
# Computing Priors
# Calculate previous goals conceded before half-time
# PHTHGC = Home Team Previous Goals Conceded Before Half Time, totaled over season
# PHTAGC = Away Team Previous Goals Conceded Before Half Time, Totaled over season

def getPreviousHalfTimeGoalConceded(data):
    teams = {}
    PHTHGC = [] 
    PHTAGC = []   
    
    # for each match
    for i in range(len(data)):
        if (i % 100000 == 0):
            for name in data.groupby('HomeTeam').mean().T.columns:
                teams[name] = [0]
                      
        HTHG = data.iloc[i]['HTHG']
        HTAG = data.iloc[i]['HTAG']

        try:
            phtgc_h = teams[data.iloc[i].HomeTeam].pop()
            phtgc_a = teams[data.iloc[i].AwayTeam].pop()
        except:
            phtgc_h = 0
            phtgc_a = 0

        PHTHGC.append(phtgc_h)
        PHTAGC.append(phtgc_a)
        phtgc_h = phtgc_h + HTAG #Home team's previous half time goals conceded before this match
        teams[data.iloc[i].HomeTeam].append(phtgc_h)
        phtgc_a = phtgc_a + HTHG #Away team's previous half time goals conceded before this match
        teams[data.iloc[i].AwayTeam].append(phtgc_a)

    data_new.loc[:,'PHTHGC'] = pd.Series(PHTHGC)
    data_new.loc[:,'PHTAGC'] = pd.Series(PHTAGC)
    return data_new

#getPreviousHalfTimeGoalConceded(data_new)

### 3.3.2 Computing previous goals conceded after half-time

In [ ]:
# Computing Priors
# Calculate previous goals conceded after half-time
# PSHHGC = Previous second half home team goals conceded, totaled over season
# PSHAGC = Previous second half away team goals conceded, totaled over season

def getPreviousSecondHalfGoalConceded(data):
    teams = {}
    PSHHGC = [] 
    PSHAGC = []   
    
    # for each match
    for i in range(len(data)):
        if (i % 100000 == 0):
            for name in data.groupby('HomeTeam').mean().T.columns:
                teams[name] = [0]
  
        FTHG = data.iloc[i]['FTHG']
        FTAG = data.iloc[i]['FTAG']   
        HTHG = data.iloc[i]['HTHG']
        HTAG = data.iloc[i]['HTAG']

        try:
            pshhgc_h = teams[data.iloc[i].HomeTeam].pop()
            pshhgc_a = teams[data.iloc[i].AwayTeam].pop()
        except:
            pshhgc_h = 0
            pshhgc_a = 0

        PSHHGC.append(pshhgc_h)
        PSHAGC.append(pshhgc_a)
        pshhgc_h = pshhgc_h + (FTAG - HTAG) #Home team's previous half time goals conceded before this match
        teams[data.iloc[i].HomeTeam].append(pshhgc_h)
        pshhgc_a = pshhgc_a + (FTHG - HTHG) #Away team's previous half time goals conceded before this match
        teams[data.iloc[i].AwayTeam].append(pshhgc_a)

    data_new.loc[:,'PSHHGC'] = pd.Series(PSHHGC)
    data_new.loc[:,'PSHAGC'] = pd.Series(PSHAGC)
    return data_new

#getPreviousSecondHalfGoalConceded(data_new)

### 3.3.3 Matches Played

In [ ]:
# Computing Priors ***NOT WORKING***
# Calculate previous goals conceded after half-time
# PMPH = Previous total matches played for home team
# PMPA = Previous total matches played for away team
def getPreviousMatchesPlayed(data):
    teams = {}
    PMPH = [] 
    PMPA = []   
    
    # for each match
    for i in range(len(data)):
        if (i % 100000 == 0):
            for name in data.groupby('HomeTeam').mean().T.columns:
                teams[name] = [0]

        try:
            pmp_h = teams[data.iloc[i].HomeTeam].pop()
            pmp_a = teams[data.iloc[i].AwayTeam].pop()
        except:
            pmp_h = 0
            pmp_a = 0

        PMPH.append(pmp_h)
        PMPA.append(pmp_a)
        pmp_h = pmp_h + 1 #Home team's previous number matches played
        teams[data.iloc[i].HomeTeam].append(pmp_h)
        pmp_a = pmp_a + 1 #Away team's previous number matches played
        teams[data.iloc[i].AwayTeam].append(pmp_a)

    data_new.loc[:,'PMPH'] = pd.Series(PMPH)
    data_new.loc[:,'PMPA'] = pd.Series(PMPA)
    return data_new

#print(getPreviousMatchesPlayed(data_new))

## 3.4 Additional Features 

### 3.4.1 Distance Travelled for Away Teams

In [ ]:
# Distance needed for away teams to travel to playing stadiums(in km)
# The locationData contains the latitude and longitude of teams
def getDistance(data, locationData):
  array = []
  for x in data.iterrows():
   
    home_lat = (locationData.loc[locationData['Team'] == x[1].HomeTeam]).Latitude
    home_long = (locationData.loc[locationData['Team'] == x[1].HomeTeam]).Longitude
    home_location = (np.float32(home_lat), np.float32(home_long))
    
    away_lat = (locationData.loc[locationData['Team'] == x[1].AwayTeam]).Latitude
   
    away_long = (locationData.loc[locationData['Team'] == x[1].AwayTeam]).Longitude
    away_location = (np.float32(away_lat), np.float32(away_long))
    array.append(np.float32(geodesic(home_location, away_location).km))
  
  
  ADIS = pd.Series(array)
  data.loc[:,'ADIS'] = ADIS

  return data

#getDistance(data, locationData)

### 3.4.2 Average shots on goal in the past 3 matches

In [ ]:
# Average shots on goal for the past 3 matches
# HAS, AAS
def getPreviousShotOnGoal_3(data):
    teams = {}
    HAS = [] 
    AAS = []   
    
    for name in data.groupby('HomeTeam').mean().T.columns:
                teams[name] = deque([None, None, None]) #[3rd, 2nd, latest data]
            
    # for each match
    for i in range(len(data)):

            
        try:
            as_h = np.mean(teams[data.iloc[i].HomeTeam])
            as_a = np.mean(teams[data.iloc[i].AwayTeam])
        except:
            as_h = None
            as_a = None

        HAS.append(as_h)
        AAS.append(as_a)

        teams[data.iloc[i].HomeTeam].popleft()
        teams[data.iloc[i].HomeTeam].append(data.iloc[i].HS)

        teams[data.iloc[i].AwayTeam].popleft()
        teams[data.iloc[i].AwayTeam].append(data.iloc[i].AS)

    data.loc[:,'HAS'] = pd.Series(HAS)
    data.loc[:,'AAS'] = pd.Series(AAS)

    return data

#getPreviousShotOnGoal_3(data)

### 3.4.3 WL Performance of past 3 matches

In [ ]:
# Performance of Home-Away teams in past 3 matches
# HM1, AM1, HM2, AM2, HM3, AM3
def getPerformanceOfLast3Matches(data):
    HM1 = []    # performance of the last match of home team
    AM1 = []    # performance of the last match of away team

    HM2 = []    # performance of the 2nd last match of home team
    AM2 = []

    HM3 = []    # performance of the 3rd last match of home team
    AM3 = []

    teams = {}
    
    for name in data.groupby('HomeTeam').mean().T.columns:
               teams[name] = deque([None, None, None])  #[3rd, 2nd, latest data]

    for i in range(len(data)):
        

        HM3.append(teams[data.iloc[i].HomeTeam].popleft())
        AM3.append(teams[data.iloc[i].AwayTeam].popleft())
        HM2.append(teams[data.iloc[i].HomeTeam][0])
        AM2.append(teams[data.iloc[i].AwayTeam][0])
        HM1.append(teams[data.iloc[i].HomeTeam][1])
        AM1.append(teams[data.iloc[i].AwayTeam][1])

        if data.iloc[i].FTR == 'H':
            teams[data.iloc[i].HomeTeam].append('W')
            teams[data.iloc[i].AwayTeam].append('L')
        elif data.iloc[i].FTR == 'A':
            teams[data.iloc[i].AwayTeam].append('W')
            teams[data.iloc[i].HomeTeam].append('L')
        else:
            teams[data.iloc[i].AwayTeam].append('D')
            teams[data.iloc[i].HomeTeam].append('D')

    data.loc[:,'HM1'] = HM1
    data.loc[:,'AM1'] = AM1
    data.loc[:,'HM2'] = HM2
    data.loc[:,'AM2'] = AM2
    data.loc[:,'HM3'] = HM3
    data.loc[:,'AM3'] = AM3

    return data

#print(getPerformanceOfLast3Matches(data))

### 3.4.4 Cumulative Full Time Goal Difference

In [ ]:
# Computing Priors
# Calculate cumulative Full-Time goal different for Home/Away teams prior to every match
# HCGD = Home Cumulative Goal Difference
# ACGD = Away Cumulative Goal Difference
def getCumulativeGoalsDiff(data):
    teams = {}
    HCGD = [] 
    ACGD = []   

    # for each match
    for i in range(len(data)):
        # as the result in 3.2.1 shows that the number of matchese per season is always the same, so here we simply use i%380==0 to check if it is a new season and to initialize the feature.
        if (i % 380 == 0):
            for name in data.groupby('HomeTeam').mean().T.columns:
                teams[name] = []

        FTHG = data.iloc[i]['FTHG']
        FTAG = data.iloc[i]['FTAG']

        try:
            cgd_h = teams[data.iloc[i].HomeTeam].pop()
            cgd_a = teams[data.iloc[i].AwayTeam].pop()
        except:
            cgd_h = 0
            cgd_a = 0

        HCGD.append(cgd_h)
        ACGD.append(cgd_a)
        cgd_h = cgd_h + FTHG - FTAG
        teams[data.iloc[i].HomeTeam].append(cgd_h)
        cgd_a = cgd_a + FTAG - FTHG
        teams[data.iloc[i].AwayTeam].append(cgd_a)

    data.loc[:,'HCGD'] = pd.Series(HCGD)
    data.loc[:,'ACGD'] = pd.Series(ACGD)

    return data

#getCumulativeGoalsDiff(data)

### 3.4.5 Goalkeeper Stats

In [ ]:
# # Goalkeeping stats for each team for season starting 2008/2009, ending at present (2021/2020)
# def getGK(data, GKData):
    
#     teams = {}
#     HSaveP = [] #Save percentage (Home team)
#     HCSP   = [] #Clean sheet percentage
#     ASaveP = [] 
#     ACSP   = [] 
    
#     ## Split data into seasons
#     data2008 = data.iloc[:379,:] # 2008-2009 season
#     data2009 = data.iloc[380:759,:]
#     data2010 = data.iloc[760:1139,:]
#     data2011 = data.iloc[1140:1519,:]
#     data2012 = data.iloc[1520:1899,:]
#     data2013 = data.iloc[1900:2279,:]
#     data2014 = data.iloc[2280:2659,:]
#     data2015 = data.iloc[2660:3039,:]
#     data2016 = data.iloc[3040:3419,:]
#     data2017 = data.iloc[3420:3799,:]
#     data2018 = data.iloc[3800:4179,:]
#     data2019 = data.iloc[4180:4559,:]
#     data2020 = data.iloc[4560:4939,:] #2020-2021 season
    
#     GKData2008 = GKData.iloc[:19,:] # 2008-2009 season
#     GKData2009 = GKData.iloc[20:39,:]
#     GKData2010 = GKData.iloc[40:59,:]
#     GKData2011 = GKData.iloc[60:79,:]
#     GKData2012 = GKData.iloc[80:99,:]
#     GKData2013 = GKData.iloc[100:119,:]
#     GKData2014 = GKData.iloc[120:139,:]
#     GKData2015 = GKData.iloc[140:159,:]
#     GKData2016 = GKData.iloc[160:179,:]
#     GKData2017 = GKData.iloc[180:199,:]
#     GKData2018 = GKData.iloc[200:219,:]
#     GKData2019 = GKData.iloc[220:239,:]
#     GKData2020 = GKData.iloc[240:259,:] #2020-2021 season

#     for i in data.groupby('HomeTeam').mean().T.columns: #for 2009 season
#             teams[i] = [] #Each team gets their own list

#     # the value corresponding to keys is a list containing the match result

#     for i in range(len(GKData2008)): # i think swap this for data instead and figure how to go through gkdata 1 at a time
#         if GKData2008.iloc[i].Squad == data2008.iloc[i].HomeTeam:
#             teams[data2008.iloc[i].HomeTeam].append(GKData2008[["SavePercentage","CSPercentage"]].iloc[i])
#     print(teams)
#     print(len(GKData2008))
        
# #         SAVEP = teams[data2009.iloc[i].HomeTeam] #use 2008 season gk data
# #         CSP = teams[data2009.iloc[i].HomeTeam]

# #         HSaveP.append(SAVEP)
# #         HCSP.append(CSP)
# #         ASaveP.append(SAVEP)
# #         ACSP.append(CSP)
        
# #         teams[data2009.iloc[i].HomeTeam].append(SAVEP)
# #         teams[data2009.iloc[i].HomeTeam].append(CSP)
# #         teams[data2009.iloc[i].AwayTeam].append(SAVEP)
# #         teams[data2009.iloc[i].AwayTeam].append(CSP)

# #         data2009.loc[:,'HSaveP'] = pd.Series(HSaveP)
# #         data2009.loc[:,'HCSP'] = pd.Series(HCSP)
# #         data2009.loc[:,'ASaveP'] = pd.Series(ASaveP)
# #         data2009.loc[:,'ACSP'] = pd.Series(ACSP)
#     return data2009

# getGK(data, GKData)

### Priors - extra features pulled

In [ ]:
def add_pickled_to_df(df,filename,column):
    matrix = pd.read_pickle(filename)
    matrix[2008] = np.NaN
#     print(matrix)
    difference = []
    for i in range(0,len(data_new)):
    #     print(ratings_matrix["mean"].loc[data_new["HomeTeam"].iloc[i]])
        if pd.isnull(matrix[year[i]].loc[df["HomeTeam"].iloc[i]]) or pd.isnull(matrix[year[i]].loc[df["AwayTeam"].iloc[i]]):
            difference.append(np.nan)
            
        else:
            difference.append(matrix[year[i]].loc[df["HomeTeam"].iloc[i]]-matrix[year[i]].loc[df["AwayTeam"].iloc[i]])

#     for i in range(0,len(difference)):
#         if difference[i]<-0.1:
#             difference[i]='A'
#         elif difference[i]>0.1:
#             difference[i]='H'
#         else:
#             difference[i]='D'

    df[column]=difference
    return df

### We have decided not to include these scraped features due to the many missing values without a reliable way to impute them.

In [ ]:
# import glob
# files = glob.glob("./Pickles/*")
# for file in files:
#     name = file.split("\\")[-1].split(".")[0].replace("DF","")
#     data_new = add_pickled_to_df(data_new,file,name)

# data_new

## 3.5 Derive Priors

In [ ]:
def DerivePriors(data_new):
    #get_previousFTResults(data_new) # dont want Full time results in the test data
    get_PreviousHTResults(data_new)
    getPreviousCumulativeGoals(data_new)
    getPreviousShots(data_new)
    getPreviousShotsOnTarget(data_new)
    getPreviousTeamFouls(data_new)
    getPreviousTeamCorners(data_new)
    getPreviousHalfTimeGoalsScored(data_new)
    getPreviousSecondHalfGoals(data_new)
    getPreviousHalfTimeGoalConceded(data_new)
    getPreviousSecondHalfGoalConceded(data_new)
    getPreviousMatchesPlayed(data_new)
    getDistance(data_new, locationData)
    getPreviousShotOnGoal_3(data_new)
    getPerformanceOfLast3Matches(data_new)
    getCumulativeGoalsDiff(data_new)
    #getGK(data, GKData)
    return data_new

In [ ]:
## Remove First Initial Season
data_new = DerivePriors(data_new).iloc[380:] #chop off first season 
y=np.delete(y,slice(0,380),axis=0)
data_new 

In [ ]:
#Final data preprocessing

# TODO:
# Implement dates using trig - done
# Add one hot encoded teams - done
# Compute custom features using priors (goals/shots on target, shots on target / total shots, home team fouls / away team fouls)
# PHGS/PHSOT, PAGS/PASOT & PHSOT/PHS, PASOT/PAS & PHTF/PATF - done
# Implement scaling but don't apply just yet - done
# Apply PCA - done

dates = data_new.iloc[:, 0:3]
month_sin = transformation(dates["Month"])[0]
month_cos = transformation(dates["Month"])[1]
week_sin = transformation(dates["Week"])[0]
week_cos = transformation(dates["Week"])[1]
day_sin = transformation(dates["Day"])[0]
day_cos = transformation(dates["Day"])[1]

teams = pd.DataFrame(home_t.toarray()).add_prefix("home_").join(pd.DataFrame(away_t.toarray()).add_prefix("away_"))

# Select only columns that contain priors, can't use in-game stats to predict the future
priors = data_new.iloc[:, 21:39]

# PHGS_PHSOT is ratio of home goals to home shots on target
PHGS_PHSOT = np.where(priors["PHSOT"] != 0, priors["PHGS"]/priors["PHSOT"], 0)
# PHGS_PHSOT is ratio of away goals to away shots on target
PAGS_PASOT = np.where(priors["PASOT"] != 0, priors["PAGS"]/priors["PASOT"], 0)
# PHSOT_PHS is ratio of home shots on target to home shots
PHSOT_PHS = np.where(priors["PHS"] != 0, priors["PHSOT"]/ (priors["PHS"] + priors["PHSOT"]), 0)
# PASOT_PAS is ratio of away shots on target to away shots
PASOT_PAS = np.where(priors["PAS"] != 0, priors["PASOT"]/ (priors["PAS"] + priors["PASOT"]), 0)
# PHTF_PATF is ratio of home fouls to away fouls
PHTF_PATF = np.where(priors["PATF"] != 0, priors["PHTF"]/priors["PATF"], 0)

# Building final dataset
X = pd.DataFrame()
X["month_cos"] = month_cos
X["month_sin"] = month_sin
X["week_cos"] = week_cos
X["week_sin"] = week_sin
X["day_cos"] = day_cos
X["day_sin"] = day_sin
X = X.join(teams).join(priors)
X["PHGS_PHSOT"] = PHGS_PHSOT.tolist()
X["PAGS_PASOT"] = PAGS_PASOT.tolist()
X["PHSOT_PHS"] = PHSOT_PHS.tolist()
X["PASOT_PAS"] = PASOT_PAS.tolist()
X["PHTF_PATF"] = PHTF_PATF.tolist()

### 3.6 Split Data

In [ ]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=False)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, shuffle=False)

y_train = np.array(y_train).reshape(len(y_train))
y_val = np.array(y_val).reshape(len(y_val))
y_test = np.array(y_test).reshape(len(y_test))
#Encode y
encoder = LabelEncoder().fit(y_train)
y_train = encoder.transform(y_train)
y_val = encoder.transform(y_val)
y_test = encoder.transform(y_test)


from tensorflow import keras
y_train_categorical = keras.utils.to_categorical(y_train)
y_val_categorical = keras.utils.to_categorical(y_val)

In [ ]:
#try without our custom features
rf, preds, all_stats_accuracy = rf_model(X_train, X_test, y_train, y_test)
print("Accuracy on all in-game stats: " + str(all_stats_accuracy) + "%")

feature_importances = feat_importances(X_train, rf)
print("Feature Importances: ")
[print('Feature: {:35} Importance: {}'.format(*pair)) for pair in feature_importances];

### 3.7 Scale data

In [ ]:
# Not sure if data needs to be scaled so just gonna leave this here
scaler = StandardScaler().fit(X_train.iloc[:, 82:])
X_train_scaled = scaler.transform(X_train.iloc[:, 82:])
X_test_scaled = scaler.transform(X_test.iloc[:, 82:])

X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)

X_train = np.array(X_train.iloc[:, 0:82])
X_test = np.array(X_test.iloc[:, 0:82])

X_train_scaled = np.hstack((X_train, X_train_scaled))
X_test_scaled = np.hstack((X_test, X_test_scaled))

# PCA
# https://towardsdatascience.com/pca-using-python-scikit-learn-e653f8989e60
# pca = PCA(0.95)
# pca.fit(X_train_scaled)
# X_train = pca.transform(X_train_scaled)
# X_test = pca.transform(X_test_scaled)

# pca = PCA(n_components=50)
# X = pca.fit_transform(X_scaled)

# from sklearn.manifold import TSNE
# X = TSNE(n_components=3, learning_rate='auto', init='random').fit_transform(X)

# from sklearn.manifold import MDS
# embedding = MDS(n_components=2)
# X = embedding.fit_transform(X) -> took way too long

# from sklearn.manifold import Isomap
# embedding = Isomap(n_components=2)
# X = embedding.fit_transform(X) -> gave terrible results

# import umap.umap_ as umap
# reducer = umap.UMAP(random_state=42,n_components=15)
# X = reducer.fit_transform(X_scaled) -> requires outdated numpy

from sklearn.decomposition import KernelPCA
kpca = KernelPCA(n_components=15, kernel='rbf')
kpca.fit(X_train_scaled)
X_train = kpca.transform(X_train_scaled)
X_test = kpca.transform(X_test_scaled)
#tune hyperparams for this -> gamma

# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# clf = LinearDiscriminantAnalysis()
# clf.fit(X_train_scaled, y_train)
# X_train = clf.transform(X_train_scaled)
# X_test = clf.transform(X_test_scaled)

# from sklearn.manifold import TSNE
# X = TSNE(n_components=2, learning_rate='auto', init='random').fit_transform(X)

#consider combos of these eg pca then lda

## 4. Methodology Overview
<a name='section4'></a>

## 4.1 Model Training & Validation
<a name='section5'></a>

In [ ]:
#Functions to remove warning to see clearer result
import warnings
from sklearn.exceptions import ConvergenceWarning
from sklearn.exceptions import UndefinedMetricWarning


warnings.filterwarnings(action='ignore', category=ConvergenceWarning)
warnings.filterwarnings(action='ignore', category=UndefinedMetricWarning)

### 4.2 Base Models

In [ ]:
## TODO: Implement into pipeline ##
# Some general comments:
# Gaussian NB is most suitable for non-categorical classification
# Based on diagram above (gaussian distributed density plots) the features we use are gaussian distributed however 
# the teams are not actually gaussian distributed 
# And the features we use are not conditionally independent as the statistics arent independent (e.g. shots affect
# shots on target etc.)
# Therefore we expect that the prediction will not be accurate and naives bayes is not suitable

#prove calculations and results later



from sklearn.naive_bayes import GaussianNB

 

gnb = GaussianNB()
y_gnb = gnb.fit(X_train, y_train).predict(X_test)
accuracy_score(y_test, y_gnb)
 

#Smoothing parameter scaling
# param_grid_nb = {
#     'var_smoothing': np.logspace(0,-9, num=100)
# }
# gnb = GridSearchCV(estimator=GaussianNB(), param_grid=param_grid_nb, verbose=1, cv=10, n_jobs=-1)
# y_gnb = gnb.fit(X_train, y_train).predict(X_test)

 

from sklearn.metrics import accuracy_score

print(accuracy_score(y_test, y_gnb), ": is the accuracy score gnb")

In [ ]:
#Using generic SVM to estimate
from sklearn.model_selection import train_test_split, KFold
from sklearn.svm import SVC, SVR

# gammas = np.power(2, np.linspace(-15, 3, 10))
# accuracy_validation = np.empty((5, len(gammas)))

# for l, gamma in enumerate(gammas):
#     svm = SVC(kernel='rbf', gamma=gamma, C=100)
#     svm.fit(X_train, y_train)
        
#     predict_test = svm.predict(X_test)  # test
#     print(accuracy_score(y_test, predict_test))

# SVM = svm.SVC(kernel="linear")   #(kernel="poly", degree=3, coef0=1, C=5) (kernel="linear")
# SVM.fit(training_data,y_train)# predict the labels on validation dataset
# predictions_SVM = SVM.predict(testing_data)# Use accuracy_score function to get the accuracy
# print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, y_test)*100)

# scores = cross_val_score(SVM, X_whole, y_enc, cv=StratifiedShuffleSplit(n_splits=10, test_size=0.2, random_state=100))
# scores.mean()

def fineTuneSVM(X_train, y_train):
    # define model and parameters
    svm = SVC()   
    # SVM solves an optimization problem of quadratic order 
    # link on SVC: https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html
    # The implementation is based on libsvm. The fit time complexity is more than quadratic with the number of samples which makes it hard to scale to dataset with more than a couple of 10000 samples.
    # Therefore, we will stick with basic kernels like linear and rbf which do the job well without sacrificing processing time.
    kernel = ['linear', 'rbf'] 
    # kernel = ['poly', 'rbf', 'sigmoid'] #Advanced kernels 
    C = [50, 10, 1.0, 0.1, 0.01]
    gamma = ['scale']
    
    # define grid search
    grid = dict(kernel=kernel,C=C,gamma=gamma)
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    grid_search = GridSearchCV(estimator=svm, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
    grid_result = grid_search.fit(X_train, y_train)
    
    # summarize results
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))

fineTuneSVM(X_train, y_train)        

In [ ]:
# Logistic Regression

from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(solver = 'liblinear',penalty = 'l1', C = 0.01)
y_lr = lr.fit(X_train, y_train).predict(X_test)
accuracy_score(y_test,y_lr)

from sklearn.metrics import accuracy_score

print(accuracy_score(y_test, y_lr), ": is the accuracy score Logistic Regression")


# Finding best hyperparameters

# define models and parameters
lr = LogisticRegression()
solvers = ['newton-cg', 'lbfgs', 'liblinear','saga']
penalty = ['l1','l2']
c_values = [100, 10, 1.0, 0.1, 0.01]
# define grid search
grid = dict(solver=solvers,penalty=penalty,C=c_values)
# cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# cv = KFold(n_splits=10, shuffle=True, random_state=1)
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=lr, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

### print all the tested results
# means = grid_result.cv_results_['mean_test_score']
# stds = grid_result.cv_results_['std_test_score']
# params = grid_result.cv_results_['params']
# for mean, stdev, param in zip(means, stds, params):
#     print("%f (%f) with: %r" % (mean, stdev, param))


### 4.3 Sophisticated Models

### 4.4 Boosting Models

In [ ]:
import sys
!{sys.executable} -m pip install xgboost


import xgboost as xgb
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

model_pipeline = [
    ('xgboost' , (Pipeline([('xgboost' ,xgb.XGBClassifier())]))), 
#     ('nn' , (Pipeline([('nn' , kears_estimator )]))), 
#     ('...' , (Pipeline([('...' , ... )]))),
#     ('...' , (Pipeline([('...' , ... )]))),
#     ('...' , (Pipeline([('...' , ... )])))
]

results = []

for pipe ,model in model_pipeline:
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    accuracy = accuracy_score(y_test, preds) * 100
    results.append(accuracy)
    output = "%s: %f" % (pipe, accuracy)
    print(output)


param_pipeline = Pipeline([("classifier", xgb.XGBClassifier())])
model_param_grid = [
                {"classifier": [xgb.XGBClassifier()],
#                  "classifier__penalty": ['l2','l1'],
#                  "classifier__C": np.logspace(0, 4, 10)
                 },
#                 {"classifier": [kears_estimator],
#                  "tfidf__ngram_range": [(1,1), (1,2), (2,2), (1,3)],
#                 "tfidf__use_idf": [True, False],
#                 "kc__epochs": [10, 100, ],
#                 "kc__dense_nparams": [32, 256, 512],
#                 "kc__init": [ 'uniform', 'zeros', 'normal', ], 
#                 "kc__batch_size":[2, 16, 32],
#                 "kc__optimizer":['RMSprop', 'Adam', 'Adamax', 'sgd'],
#                 "kc__dropout": [0.5, 0.4, 0.3, 0.2, 0.1, 0]
#                  },
#                 {"classifier": [...],
#                  "classifier__...": [...],
#                  },
#                 {"classifier": [...],
#                  "classifier__...": [...],
#                  },
#                 {"classifier": [...],
#                  "classifier__...": [...],
#                  }
                ]
# gridsearch = GridSearchCV(param_pipeline, model_param_grid, cv=5, verbose=1,n_jobs=-1, return_train_score=True)
# best_model = gridsearch.fit(X_train,y_train)
# print(best_model.best_estimator_)
# print("The mean accuracy of the model is:",best_model.score(X_test,y_test))

In [ ]:
# Optimising hyperparameters for XGBoost
import xgboost as xgb

#xgb_cl = xgb.XGBClassifier(use_label_encoder=False)
#xgb_cl.fit(X_train, y_train)

# Round 1 values inspired by https://towardsdatascience.com/beginners-guide-to-xgboost-for-classification-problems-50f75aac5390
#param_grid = {
#    "max_depth": [3, 4, 5, 7],
#    "learning_rate": [0.1, 0.01, 0.05],
#    "gamma": [0, 0.25, 1],
#    "reg_lambda": [0, 1, 10],
#    "scale_pos_weight": [1, 3, 5],
#    "subsample": [0.8],
#    "colsample_bytree": [0.5],
#}

# Init Grid Search
#grid_cv = GridSearchCV(xgb_cl, param_grid, n_jobs=-1, cv=3, scoring="roc_auc")

# Fit
#_ = grid_cv.fit(X_train, y_train)

# Round 1 output:
#{'colsample_bytree': 0.5,
# 'gamma': 0,
# 'learning_rate': 0.1,
# 'max_depth': 3,
# 'reg_lambda': 0,
# 'scale_pos_weight': 1,
# 'subsample': 0.8}

# Round 2:
#param_grid = {
#    "max_depth": [1, 2, 3],
#    "learning_rate": [0.1, 0.15, 0.2],
#    "gamma": [0, 0.01, 0.02],
#    "reg_lambda": [0, 1, 10],
#    "scale_pos_weight": [1, 3, 5],
#    "subsample": [0.8],
#    "colsample_bytree": [0.5],
#}

#grid_cv_2 = GridSearchCV(xgb_cl, param_grid, n_jobs=-1, cv=3, scoring="roc_auc")

#_ = grid_cv_2.fit(X_train, y_train)

#grid_cv_2.best_params_

# Round 2 output:
#{'colsample_bytree': 0.5,
# 'gamma': 0,
# 'learning_rate': 0.1,
# 'max_depth': 1,
# 'reg_lambda': 0,
# 'scale_pos_weight': 1,
# 'subsample': 0.8}

# Don't think another round is necessary, now to compute accuracy using these hyperparameters
final_cl = xgb.XGBClassifier(
    colsample_bytree=0.5,
    subsample=0.8,
    gamma=0,
    learning_rate=0.1,
    max_depth=1,
    reg_lambda=0,
    use_label_encoder=False,
    eval_metric='mlogloss'
)

_ = final_cl.fit(X_train, y_train)

preds = final_cl.predict(X_test)

accuracy_score(y_test, preds)

# Accuracy with new hyperparameters is: 0.4824561403508772, accuracy with default hyperparameters is 0.42172740

In [ ]:
# Optimising hyperparameters for AdaBoost
# First classified boosting algorithm

# Sources used:
# https://towardsdatascience.com/adaboost-from-scratch-37a936da3d50
# https://analyticsindiamag.com/introduction-to-boosting-implementing-adaboost-in-python/
# https://machinelearningmastery.com/adaboost-ensemble-in-python/

# Hyperparameter types (Modified Y/N):
# Num. of trees (Y)
# Weak learner (N)
# Learning rate (Y)
# Alternate algorithm (Decision Tree/Logistic Regression)

#First classified boosting algorithm

from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression

# define the model
abc = AdaBoostClassifier()
abc.fit(X_train, y_train)

# grid = {
#     "n_estimators": [10, 50, 100, 500],
#     "learning_rate": [0.0001, 0.001, 0.01, 0.1, 1.0],
# }
# abc_grid_cv = GridSearchCV(estimator=abc, param_grid = grid, n_jobs=-1, cv=3, scoring="accuracy")

# abc_grid_cv.fit(X_train, y_train)

# adc_pred = abc_grid_cv.predict(X_test)

# print("Train: %f , Params: %s, Test: %f" % (abc_grid_cv.best_score_, abc_grid_cv.best_params_, accuracy_score(y_test, adc_pred)))

#Best score and parameters for decision tree round 1
# Train: 0.525740 , Params: {'learning_rate': 0.001, 'n_estimators': 500}, Test: 0.521592

#Round 2
grid2 = {
    "n_estimators": [400, 500, 600, 800, 1000],
    "learning_rate": [0.0008, 0.001, 0.0012, 0.0014],
}
abc_grid_cv2 = GridSearchCV(estimator=abc, param_grid = grid2, n_jobs=-1, cv=3, scoring="accuracy")
abc_grid_cv2.fit(X_train, y_train)
abc_pred2 = abc_grid_cv2.predict(X_test)
print("Train: %f , Params: %s, Test: %f" % (abc_grid_cv2.best_score_, abc_grid_cv2.best_params_, accuracy_score(y_test, abc_pred2)))

#Best score and parameters for decision tree round 2
#Train: 0.525740 , Params: {'learning_rate': 0.0008, 'n_estimators': 400}, Test: 0.521592
#No improvement use as final values

# Try with logistic regression algorithm
# abc_lr = AdaBoostClassifier(base_estimator=LogisticRegression(solver = 'liblinear',penalty = 'l1', C = 0.01))
# abc_lr.fit(X_train, y_train)

# grid = {
#     "n_estimators": [10, 50, 100, 500],
#     "learning_rate": [0.0001, 0.001, 0.01, 0.1, 1.0],
# }
# abc_lr_grid_cv = GridSearchCV(estimator=abc_lr, param_grid = grid, n_jobs=-1, cv=3, scoring="accuracy")

# abc_lr_grid_cv.fit(X_train, y_train)

# adc_lr_pred = abc_lr_grid_cv.predict(X_test)

# print("Train: %f , Params: %s, Test: %f" % (abc_lr_grid_cv.best_score_, abc_lr_grid_cv.best_params_, accuracy_score(y_test, adc_lr_pred)))

#Output
# Train: 0.489301 , Params: {'learning_rate': 0.1, 'n_estimators': 500}, Test: 0.486505

#Use round 2 decision tree output as predictor
abc_final = AdaBoostClassifier(n_estimators=400, learning_rate=0.0008)



In [ ]:
# Optimising hyperparameters for GradientBoost

# Sources used:
# https://www.datasciencelearner.com/gradient-boosting-hyperparameters-tuning/
# https://www.analyticsvidhya.com/blog/2016/02/complete-guide-parameter-tuning-gradient-boosting-gbm-python/

from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor

#define the model
# gbc = GradientBoostingClassifier(subsample = 0.8)
# gbc.fit(X_train, y_train)
# gbc_pre_pred = gbc.predict(X_test)
# print("Pretuning Test: %f" % accuracy_score(y_test, gbc_pre_pred))

# grid = {
#     "n_estimators":[5,50,250,500],
#     "max_depth":[5,6],
#     "learning_rate":[0.1],
#     "min_samples_split":[40]
# }

# gbc_grid_cv = GridSearchCV(estimator=gbc, param_grid = grid, n_jobs=-1, cv=3, scoring="accuracy")
# gbc_grid_cv.fit(X_train, y_train)
# gbc_pred = gbc_grid_cv.predict(X_test)
# print("Train: %f , Params: %s, Test: %f" % (gbc_grid_cv.best_score_, gbc_grid_cv.best_params_, accuracy_score(y_test, gbc_pred)))

# Outputs
# Train: 0.515040 , Params: {'learning_rate': 0.1, 'max_depth': 5, 'min_samples_split': 40, 'n_estimators': 5}, Test: 0.522267

#Round 2
grid2 = {
    "n_estimators":[3,5,7,9],
    "max_depth":[5],
    "learning_rate":[0.05,0.1,0.2],
    "min_samples_split":[30,40,50]
}

gbc_grid_cv2 = GridSearchCV(estimator=gbc, param_grid = grid2, n_jobs=-1, cv=3, scoring="accuracy")
gbc_grid_cv2.fit(X_train, y_train)
gbc_pred2 = gbc_grid_cv2.predict(X_test)
print("Train: %f , Params: %s, Test: %f" % (gbc_grid_cv2.best_score_, gbc_grid_cv2.best_params_, accuracy_score(y_test, gbc_pred2)))

# Outputs
# Train: 0.525448 , Params: {'learning_rate': 0.1, 'max_depth': 5, 'min_samples_split': 50, 'n_estimators': 7}, Test: 0.528340

gbc_final = GradientBoostingClassifier(
    subsample = 0.8, 
    learning_rate = 0.1, 
    max_depth = 5, 
    min_samples_split = 50, 
    n_estimators = 7
)

In [ ]:
# LightGBM
# https://towardsdatascience.com/kagglers-guide-to-lightgbm-hyperparameter-tuning-with-optuna-in-2021-ed048d9838b5

#import sys
#!{sys.executable} -m pip install optuna
#!{sys.executable} -m pip install lightgbm
import lightgbm
import optuna
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold


def objective(trial, X, y):
    param_grid = {
        # "device_type": trial.suggest_categorical("device_type", ['gpu']),
        "n_estimators": trial.suggest_categorical("n_estimators", [10000]),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "num_leaves": trial.suggest_int("num_leaves", 20, 3000, step=20),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 200, 10000, step=100),
        "lambda_l1": trial.suggest_int("lambda_l1", 0, 100, step=5),
        "lambda_l2": trial.suggest_int("lambda_l2", 0, 100, step=5),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0, 15),
        "bagging_fraction": trial.suggest_float(
            "bagging_fraction", 0.2, 0.9, step=0.1
        ),
        "bagging_freq": trial.suggest_categorical("bagging_freq", [1]),
        "feature_fraction": trial.suggest_float(
            "feature_fraction", 0.2, 0.9, step=0.1
        ),
    }

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    cv_scores = np.empty(5)
    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        model = lightgbm.LGBMClassifier(objective="multiclass", **param_grid)
        model.fit(
            X_train,
            y_train,
            eval_set=[(X_test, y_test)],
            eval_metric="multi_logloss",
            early_stopping_rounds=100,
        )
        preds = model.predict_proba(X_test)
        cv_scores[idx] = log_loss(y_test, preds)

    return np.mean(cv_scores)


study = optuna.create_study(direction="minimize", study_name="LGBM Classifier")
func = lambda trial: objective(trial, X, y)
study.optimize(func, n_trials=20)

print(f"\tBest value (rmse): {study.best_value:.5f}")

print(f"\tBest params:")
for key, value in study.best_params.items():
    print(f"\t\t{key}: {value}")

model = lightgbm.LGBMClassifier(objective="multiclass", **study.best_params)
model.fit(X_train, y_train, eval_set=[(X_test,y_test), (X_train, y_train)], eval_metric='multi_logloss')

print('Training accuracy ' + str(model.score(X_train, y_train)))
print('Testing accuracy ' + str(model.score(X_test, y_test)))


### 4.5 Neural Network Models

In [ ]:
#Creating Standard, baseline NN
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers import Dense, Input, Dropout
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from keras.callbacks import EarlyStopping

#need to stop randomization
batch_size = 50
epochs = 5
#Rename to vanilla
model = Sequential()
#need to replace input shape by X shape
model.add(layers.Dense(128, input_shape=(X_train.shape[1],)))
model.add(layers.Activation('linear'))
model.add(Dropout(0.1), )
model.add(layers.Dense(3))
model.add(layers.Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

history = model.fit(X_train, y_train_categorical, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_val, y_val_categorical))

preds = model.predict(X_test)
preds = np.argmax(preds, axis=1)
accuracy = accuracy_score(y_test, preds) * 100
print(accuracy)

In [ ]:
#Hyperparam tuning for vanilla nn

#trial activation function
#trial optimiser
#trial number of neurons

# import sys
# !{sys.executable} -m pip install optuna
import optuna

def tune(objective):
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=100)

    params = study.best_params
    best_score = study.best_value
    print(f"Best score: {best_score}\n")
    print(f"Optimized parameters: {params}\n")
    return params

history = []

def ridge_objective(trial):
#     clear_session()
    
    model = Sequential()
    model.add(layers.Dense(trial.suggest_categorical('n_nodes', [32, 64, 128, 256]), input_shape=(X_train.shape[1],)))
    model.add(layers.Activation(trial.suggest_categorical('activation', ['relu', 'linear', 'tanh'])))
    model.add(Dropout(0.1), )
    model.add(layers.Dense(3))
    model.add(layers.Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=trial.suggest_categorical('optimizer',['adam','rmsprop','adagrad', 'sgd']), metrics=['accuracy'])
    
#     stopping = EarlyStopping(monitor='val_acc', patience=50)
    #what does using validation_split or validation_data do here exactly?
    history = model.fit(X_train, y_train_categorical, batch_size=50, epochs=5, validation_split=0.1, shuffle=False)
    return model.evaluate(X_val, y_val_categorical)[1]

ridge_params = tune(ridge_objective)
#this is a dictionary : Optimized parameters: {'n_nodes': 128, 'activation': 'linear', 'optimizer': 'adagrad'}
#can just extract params to make new model -> consider deleting old model or placing it after this
# ridge = Ridge(**ridge_params, random_state=42)

In [ ]:
#Creating Deep NN
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers import Dense, Input, Dropout
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from keras.callbacks import EarlyStopping

#need to stop randomization
batch_size = 50
epochs = 2
#Rename to vanilla
model = Sequential()
#need to replace input shape by X shape
model.add(layers.Dense(128, input_shape=(X_train.shape[1],)))
model.add(layers.Activation('relu'))
# model.add(Dropout(0.1), )
model.add(layers.Dense(64, input_shape=(X_train.shape[1],)))
model.add(layers.Activation('relu'))
# model.add(Dropout(0.1), )
model.add(layers.Dense(32, input_shape=(X_train.shape[1],)))
model.add(layers.Activation('relu'))
# model.add(Dropout(0.1), )
model.add(layers.Dense(3))
model.add(layers.Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

history = model.fit(X_train, y_train_categorical, batch_size=batch_size, epochs=epochs, verbose=1, validation_split=0.1)

preds = model.predict(X_test)
preds = np.argmax(preds, axis=1)
accuracy = accuracy_score(y_test, preds) * 100
print(accuracy)

In [ ]:
#Creating RNN #######rename variables appropriately
np.random.seed(42)
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, SimpleRNN

batch_size = 1
# batch_size = 100
X_train_lstm = np.array(X_train).reshape(-1, batch_size, X_train.shape[1])
y_train_categorical_lstm = np.array(y_train_categorical).reshape(-1, batch_size, 3)
X_test_lstm = np.array(X_test).reshape(-1, batch_size, X_train.shape[1])
#Rename to LSTM
model = Sequential()
model.add(SimpleRNN(128, input_shape = (13, 15), return_sequences = True))
model.add(Dropout(0.1), )
model.add(layers.Dense(3))
model.add(layers.Activation('sigmoid'))
model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ['accuracy'])

history = model.fit(X_train_test, y_train_test, epochs = 20)

preds = model.predict(X_test_test)
preds = preds.reshape(1482, 3)
preds = np.argmax(preds, axis=1)
accuracy = accuracy_score(y_test, preds) * 100
print(accuracy)

In [ ]:
#Creating GRU #######rename variables appropriately
np.random.seed(42)
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, GRU

batch_size = 1
# batch_size = 100
X_train_lstm = np.array(X_train).reshape(-1, batch_size, X_train.shape[1])
y_train_categorical_lstm = np.array(y_train_categorical).reshape(-1, batch_size, 3)
X_test_lstm = np.array(X_test).reshape(-1, batch_size, X_train.shape[1])
#Rename to LSTM
model = Sequential()
model.add(GRU(128, input_shape = (13, 15), return_sequences = True))
model.add(Dropout(0.4), )
model.add(layers.Dense(3))
model.add(layers.Activation('sigmoid'))
model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ['accuracy'])

history = model.fit(X_train_test, y_train_test, epochs = 20)


preds = model.predict(X_test_test)
preds = preds.reshape(1482, 3)
preds = np.argmax(preds, axis=1)
accuracy = accuracy_score(y_test, preds) * 100
print(accuracy)

In [ ]:
#Creating LSTM
np.random.seed(42)
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM

batch_size = 1
# batch_size = 100

X_train_test = np.reshape(X_train, (266, 13, 15))

y_train_test = np.reshape(y_train_categorical, (266, 13, 3))

X_test_test = X_test.reshape(114, 13, 15)

# X_train_lstm, y_train_categorical_lstm = lstm_data_transform(X_train, y_train_categorical, num_steps=5)
# X_train_lstm = np.array(X_train_lstm).reshape(-1, 5, X_train.shape[1])
# y_train_categorical_lstm = np.array(y_train_categorical_lstm).reshape(-1, 5, y_train_categorical.shape[1])

#Rename to LSTM
model = Sequential()
model.add(LSTM(128, input_shape = (13, 15), return_sequences = True))
model.add(Dropout(0.4), )
model.add(layers.Dense(3))
model.add(layers.Activation('sigmoid'))
model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ['accuracy'])

history = model.fit(X_train_test, y_train_test, epochs = 50, batch_size = 100)

preds = model.predict(X_test_test)
preds = preds.reshape(1482, 3)
preds = np.argmax(preds, axis=1)
accuracy = accuracy_score(y_test, preds) * 100
print(accuracy)

In [ ]:
#Creating CNN #######rename variables appropriately
np.random.seed(42)
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv1D

batch_size = 1
# batch_size = 100
X_train_lstm = np.array(X_train).reshape(-1, batch_size, X_train.shape[1])
y_train_categorical_lstm = np.array(y_train_categorical).reshape(-1, batch_size, 3)
X_test_lstm = np.array(X_test).reshape(-1, batch_size, X_train.shape[1])
#Rename to LSTM
model = Sequential()
# input_shape = (1, X_train.shape[1])
model.add(Conv1D(128, kernel_size=X_train.shape[1], input_shape=(None, 1)))
# model.add(Conv1D(64, kernel_size=X_train.shape[1], input_shape=(None, 1)))
# model.add(Dropout(0.1), )
model.add(layers.Dense(3))
model.add(layers.Activation('sigmoid'))
model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ['accuracy'])

history = model.fit(X_train, y_train_categorical_lstm, epochs = 20)

preds = model.predict(X_test)
preds = np.argmax(preds, axis=2)
accuracy = accuracy_score(y_test, preds) * 100
print(accuracy)

### 4.6 Time Series Models

In [ ]:
# Prophet
# https://machinelearningmastery.com/time-series-forecasting-with-prophet-in-python/

from fbprophet import Prophet
from fbprophet.diagnostics import cross_validation, performance_metrics

def objective(trial, df):
    seasonality = ['additive', 'multiplicative']
    param_grid = {
        "changepoint_prior_scale": trial.suggest_uniform("changepoint_prior_scale", 0.001, 0.5),
        "seasonality_prior_scale": trial.suggest_uniform("seasonality_prior_scale", 0.01, 10),
        "seasonality_mode": seasonality[trial.suggest_int("seasonality_mode", 0, 1)]
    }

    m = Prophet(**param_grid)
    m.fit(df)
    df_cv = cross_validation(m, 
                             initial='3458 days', 
                             period='13 days', 
                             horizon = '13 days',
                             parallel="processes")
    
    df_p = performance_metrics(df_cv, rolling_window=1)
    
    return df_p['rmse'].values[0]


ds = pd.to_datetime(data["Date"])

encoder = LabelEncoder().fit(y)

prophet_df = pd.DataFrame()
prophet_df["ds"] = ds
prophet_df["y"] = encoder.transform(y)

#study = optuna.create_study()
#func = lambda trial: objective(trial, prophet_df)
#study.optimize(func, n_trials=20)

#print(f"\tBest value (rmse): {study.best_value}")

#print(f"\tBest params:")
#for key, value in study.best_params.items():
#    print(f"\t\t{key}: {value}")

#Best value (rmse): 0.8722181615529774
#Best params:
#changepoint_prior_scale: 0.3742218538948863
#seasonality_prior_scale: 0.05051527961102975
#seasonality_mode: 1 (multiplicative)

best_params = {
    "changepoint_prior_scale": 0.3742218538948863,
    "seasonality_prior_scale": 0.05051527961102975,
    "seasonality_mode": 'multiplicative'
}

model = Prophet(**best_params)

# 3458 is the equivalent of 266 seasons if we define a season as 13 matches and reserve the remaining 114 seasons for testing
ds_train = prophet_df.head(3458)
ds_test = prophet_df.tail(len(prophet_df) - 3458)

model.fit(ds_train)

future = ds_test.drop(columns=["y"])

forecast = model.predict(future)

#print(np.rint(forecast["yhat"]))
accuracy = accuracy_score(ds_test["y"], np.rint(forecast["yhat"]))
print(accuracy)

# Accuracy as of 2021-12-07 is 0.22739541160593793

In [ ]:
# Arima

from statsmodels.tsa.statespace.sarimax import SARIMAX
import warnings
import optuna
import pmdarima as pm

def objective(trial, X):
    param_grid = (
        trial.suggest_int("p", 1, 3),
        0,
        trial.suggest_int("q", 1, 3)
    )
    
    param_grid_seasonal = (
        trial.suggest_int("P", 0, 3),
        1,
        trial.suggest_int("Q", 0, 3),
        12
    )

    train_size = int(len(X) * 0.70)
    train, test = X[0:train_size], X[train_size:]
    history = [x for x in train]
    # make predictions
    predictions = list()
    for t in range(len(test)):
        model = SARIMAX(history, order=param_grid, seasonal_order=param_grid_seasonal)
        model_fit = model.fit()
        yhat = model_fit.forecast()[0]
        predictions.append(yhat)
        history.append(test[t])
    # calculate out of sample error
    rmse = sqrt(mean_squared_error(test, predictions))
    return rmse

def evaluate_arima_model(X, arima_order):
    # prepare training dataset
    train_size = int(len(X) * 0.66)
    train, test = X[0:train_size], X[train_size:]
    history = [x for x in train]
    # make predictions
    predictions = list()
    for t in range(len(test)):
        model = ARIMA(history, order=arima_order)
        model_fit = model.fit()
        yhat = model_fit.forecast()[0]
        predictions.append(yhat)
        history.append(test[t])
    # calculate out of sample error
    rmse = sqrt(mean_squared_error(test, predictions))
    return rmse

# evaluate combinations of p, d and q values for an ARIMA model
def evaluate_models(dataset, p_values, d_values, q_values):
    dataset = dataset.astype('float32')
    best_score, best_cfg = float("inf"), None
    for p in p_values:
        for d in d_values:
            for q in q_values:
                order = (p,d,q)
                try:
                    rmse = evaluate_arima_model(dataset, order)
                    if rmse < best_score:
                        best_score, best_cfg = rmse, order
                    print('ARIMA%s RMSE=%.3f' % (order,rmse))
                except:
                    continue
    print('Best ARIMA%s RMSE=%.3f' % (best_cfg, best_score))
    
def get_accuracy(X):
    train_size = int(len(X) * 0.70)
    train, test = X[0:train_size], X[train_size:]
    history = [x for x in train]
    # make predictions
    predictions = list()
    for t in range(len(test)):
        model = SARIMAX(history)
        model_fit = model.fit()
        yhat = model_fit.forecast()[0]
        predictions.append(round(yhat))
        history.append(test[t])
    return accuracy_score(test, predictions)


idx = data['Date']
encoder = LabelEncoder().fit(y)
y = encoder.transform(y)
ts = pd.Series(y, index=idx).astype('float32')

#p_values = [0, 1, 2, 4, 6, 8, 10]
#d_values = range(0, 3)
#q_values = range(0, 3)

warnings.filterwarnings("ignore")

#evaluate_models(ts.values, p_values, d_values, q_values)

#study = optuna.create_study()
#func = lambda trial: objective(trial, ts.values)
#study.optimize(func, n_trials=10)

#print(f"\tBest value (rmse): {study.best_value}")

#print(f"\tBest params:")
#for key, value in study.best_params.items():
#    print(f"\t\t{key}: {value}")

#smodel = pm.auto_arima(ts.values, start_p=1, start_q=1,
#                         test='adf',
#                         max_p=3, max_q=3, m=12,
#                         start_P=0, seasonal=True,
#                         d=None, D=1, trace=True,
#                         error_action='ignore',  
#                         suppress_warnings=True, 
#                         stepwise=True)

#print(smodel.summary())

# Summary:
# Optuna - took 2 hours and didn't return any hyperparameters (never finished)
# pmd.auto_arima - ran out of memory
# Standard grid search - same as Optuna

print(get_accuracy(ts))

# Accuracy as of 2021-12-12 is 0.2813765182186235

## 5. Results
<a name='section6'></a>

## 6. Final Predictions on Test Set
<a name='section7'></a>